In [1]:
import cv2
from cv2 import VideoCapture
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import os

objc[90764]: Class CaptureDelegate is implemented in both /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1615824e8) and /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x143648860). One of the two will be used. Which one is undefined.
objc[90764]: Class CVWindow is implemented in both /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x161582538) and /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1342f0a68). One of the two will be used. Which one is undefined.
objc[90764]: Class CVView is implemented in both /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x161582560) and /Users/sahaj/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1342f0a90). One of the two will be used. Which

In [2]:
#path or np arrays
DATA_PATH = os.path.join('MP_Data')
#actions =  np.array(['Hello','ILoveYou','Language','ThankYou','Namaste','sorry'])
#actions =  np.array(['Hello','I','Indian','Language','ThankYou']) #action1
#actions =  np.array(['ILoveYou','Hello','I','ThankYou','Language']) #fa5
#actions =  np.array(['Hello','ILoveYou','I','Language','ThankYou','Indian']) #T6
actions =  np.array(['Idle','ILoveYou','Hello','I','ThankYou','Language']) #Totla

no_sequence = 30
sequence_lenth=30

label_map = {label:num for num,label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_lenth):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

print(label_map)

{'Idle': 0, 'ILoveYou': 1, 'Hello': 2, 'I': 3, 'ThankYou': 4, 'Language': 5}


In [3]:

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Metal device set to: Apple M2


2023-03-27 14:48:08.840611: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-27 14:48:08.840643: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model.load_weights('I6.h5')

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245),(0,245,16),(0,0,255),(255,0,255),(255,158,255)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        #cv2.putText(output_frame, (actions[num]+str(prob*100)), (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2, cv2.LINE_AA)
 
    return output_frame

In [5]:
# 1. New detection variables
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# mujhe add karna pada tha size kam karne ke liye

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)
    return image , results

def draw_landmarks(image , results):
    mp_drawing.draw_landmarks(image,results.face_landmarks , mp_holistic.FACEMESH_CONTOURS, mp_drawing.DrawingSpec( color=(80,110,10), thickness=1 ,circle_radius=1), mp_drawing.DrawingSpec( color=(80,256,121), thickness=1 ,circle_radius=1))
    #mp_drawing.draw_landmarks(image,results.face_landmarks , mp_holistic.CONTOURS, mp_drawing.DrawingSpec( color=(80,110,10), thickness=1 ,circle_radius=1), mp_drawing.DrawingSpec( color=(80,256,121), thickness=1 ,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks , mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec( color=(80,22,10), thickness=2 ,circle_radius=4), mp_drawing.DrawingSpec( color=(80,44,121), thickness=2 ,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec( color=(121,11,250), thickness=2 ,circle_radius=4), mp_drawing.DrawingSpec( color=(121,89,76), thickness=2 ,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec( color=(245,117,66), thickness=2 ,circle_radius=4), mp_drawing.DrawingSpec( color=(245,56,210), thickness=2 ,circle_radius=2))

def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose,face,lh,rh])

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8
pr=0
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        image =cv2.flip(image ,1)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold and actions[np.argmax(res)] != 'Idle': 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            #print(res)
                    else:
                        sentence.append(actions[np.argmax(res)])
                        #print(res)

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        #image =cv2.flip(image ,1)
        cv2.rectangle(image, (0,0), (2000, 80), (48, 44, 44), -1)
        cv2.putText(image, ' '.join(sentence), (8,60), 
                       cv2.FONT_HERSHEY_TRIPLEX, 2, (17,191, 252), 1, cv2.LINE_AA)
        
        # Show to screen
        if(max(res)>.8):
            
            if(max(res)>=pr):
                pr=max(res)
            else:
                print((actions[np.argmax(res)]+str(pr)))
                pr=0
        #resize = ResizeWithAspectRatio(image, width=1000) #mujhe add karna pada tha
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-03-27 14:48:30.896914: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-27 14:48:31.012318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 389ms/step
Idle2.75978946685791
1/1 [==============================] - 0s 158ms/step
Idle0.999964
1/1 [==============================] - 0s 159ms/step
Idle0.9999571
1/1 [==============================] - 0s 196ms/step
Idle0.9998752
1/1 [==============================] - 0s 152ms/step
Idle0.99986076
1/1 [==============================] - 0s 225ms/step
Idle0.9989911
1/1 [==============================] - 0s 156ms/step
Idle0.98886275
1/1 [==============================] - 0s 157ms/step
Hello1.0
1/1 [==============================] - 0s 164ms/step
Idle0.99194604
1/1 [==============================] - 0s 155ms/step
Idle0.985168
1/1 [==============================] - 0s 157ms/step
Idle0.99999034
1/1 [==============================] - 0s 161ms/step
Idle0.99999106
1/1 [==============================] - 0s 169ms/step
Idle0.99999166
1/1 [==============================] - 0s 144ms/step
Idle0.9999925
1/1 [==============================] - 0s 146ms/step
Idl

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()